# Chapter 10: Modeling Relationships #

## Code segment 10.1
Figure 10.1 in Section 10.1

In [2]:
#Code Segment 10.1
#Show Fisher z-transform
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.dpi']= 300

def fisherz(rho):
    #Fisher z-transform
    return(.5*np.log((1+rho)/(1-rho)))

x=np.arange(-.99,.9999,.01)
plt.plot(x,fisherz(x),label="transform")

fzeq1=(np.exp(2)-1)/(np.exp(2)+1)  #argument where z-transform equals one
plt.plot(x,x/fzeq1,label="linear to z=1")
plt.title("Figure 10.1: Fisher z-transform")
plt.xlabel("Correlation")
plt.ylabel("Fisher Z")
plt.legend()
plt.grid()
plt.show();

## Code segment 10.2
Pearson/Spearman calculations in Section 10.2

In [3]:
#Code Segment 10.2
import scipy.stats as spst
#Form ranks from Pearson example
x=[1,-.1,1.5,1.7,-1,5]
y=[1,-.1,1.5,1.7,-1,-2]

print("X data:",x)
xr=7-spst.rankdata(x)
print("X ranks:",xr)

print("Y data:",y)
yr=7-spst.rankdata(y)
print("Y ranks:",yr)

pearson,psig=spst.pearsonr(x,y)
#Compute Spearman by doing Pearson on ranks
spearman,ssig=spst.pearsonr(xr,yr)
#Direct call to Spearmsn
spear_scipy,sspsig=spst.spearmanr(x,y)

print("Pearson correlation:",pearson)
print("Spearman correlation from Pearson ranks:",spearman)
print("Spearman from scipy:",spear_scipy)

X data: [1, -0.1, 1.5, 1.7, -1, 5]
X ranks: [4. 5. 3. 2. 6. 1.]
Y data: [1, -0.1, 1.5, 1.7, -1, -2]
Y ranks: [3. 4. 2. 1. 5. 6.]
Pearson correlation: -0.2844419046262926
Spearman correlation from Pearson ranks: 0.14285714285714285
Spearman from scipy: 0.14285714285714288


## Code segment 10.3
Acquire regional stock market indices in Section 10.3

In [4]:
#Code Segment 10.3
import pandas as pd
import numpy as np
#Get regional stock market index data from Ken French's website.
#Convert daily to Wednesday-Wednesday weekly.

ff_head='http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/'
ff_foot="_3_Factors_Daily_CSV.zip"
ff_names=["Europe","North_America","Japan"]

for name_index in range(len(ff_names)):
    print("Inputting ",ff_names[name_index])
    ffurl=ff_head+ff_names[name_index]+ff_foot
    #Skip the header rows
    df_region = pd.read_csv(ffurl, skiprows=6)
    #Standardize name of Date column and market return column
    col0=df_region.columns[0]
    df_region.rename(columns={col0:'Date'},inplace=True)
    df_region.rename(columns={"Mkt-RF":ff_names[name_index]},inplace=True)
    #Merge into aggregate
    if name_index == 0:
        df_returns=df_region[df_region.columns[0:2]]
    else:
        df_returns = df_returns.merge(df_region[df_region.columns[0:2]], 
                            left_on='Date', right_on='Date')

#Convert to log-returns
df_logs_day=np.log(1+df_returns[df_returns.columns[1:]]/100)

#Convert dates to datetime format
df_logs_day.insert(0,"Date",df_returns["Date"],True)
df_logs_day["Date"] = pd.to_datetime(df_logs_day["Date"], format='%Y%m%d')
        
#Convert log-returns to weekly (Wednesday-Wednesday)
#to avoid asynchronous trading effects
df_logs_day = df_logs_day.set_index("Date")
df_logs=df_logs_day.resample('W-Wed').sum()
#(Will include some holidays like July 4 and December 25, so a little off)

#Remove the partial year at the end
lastyear=df_logs.index[-1].year
df_logs.drop(df_logs.index[df_logs.index.year==lastyear],axis=0,inplace=True)

periodicity=52   #For use in later code segments

nobs=len(df_logs)
print(nobs," weekly observations starting",df_logs.index[0].strftime("%Y-%m-%d"), \
      "ending",df_logs.index[-1].strftime("%Y-%m-%d"))

Inputting  Europe


Inputting  North_America


Inputting  Japan


1696  weekly observations starting 1990-07-04 ending 2022-12-28


## Code segment 10.4
Regional stock market index correlation matrix in Section 10.3

In [5]:
#Code Segment 10.4
#Get and show correlation matrix and
#standard deviations
corr_matrix=df_logs[df_logs.columns].corr()
cov_matrix=df_logs[df_logs.columns].cov()
std_devs=[]
for i in range(len(ff_names)):
    #Annualize weekly data
    std_devs.append(np.sqrt(periodicity*cov_matrix.iloc[i,i]))
    
print("Correlation matrix and standard deviations (10.8):")
print(corr_matrix)
print('Annualized standard deviations:\n',std_devs)
zsig=np.sqrt(1/(nobs-3))
rsig=(np.exp(2*zsig)-1)/(np.exp(2*zsig)+1)
print('Correlation significance:',rsig)

Correlation matrix and standard deviations (10.10):
                 Europe  North_America     Japan
Europe         1.000000       0.761644  0.510496
North_America  0.761644       1.000000  0.412393
Japan          0.510496       0.412393  1.000000
Annualized standard deviations:
 [0.1857710991004088, 0.1690650344540889, 0.20216641309990663]
Correlation significance: 0.024298867052249755


## Code segment 10.5
Regional stock market index global minimum variance portfolio in Section 10.3

In [6]:
#Code Segment 10.5
#Compute minimum variance portfolio of three regions (4.8)
cov_matrix_inverse=pd.DataFrame(np.linalg.pinv(cov_matrix.values), \
            cov_matrix.columns,cov_matrix.index)
u=pd.Series([1]*len(cov_matrix_inverse),index=cov_matrix_inverse.index)
minvport=cov_matrix_inverse.dot(u)
minvar=1/minvport.dot(u)  #This is second part of formula (4.8)
minvport*=minvar    #This is first part of formula (4.8)
print('Minimum variance portfolio:')
print(minvport.to_string())

#Annualized standard deviation
annminstd=np.sqrt(minvar*periodicity)
print('Minimum variance portfolio annualized std deviation:',annminstd)
#Find minimum component standard deviation
compminstd=min(std_devs)
comp_index=std_devs.index(compminstd)
print("Lowest component annualized std deviation",compminstd,"(",df_logs.columns[comp_index],")")

Minimum variance portfolio:
Europe           0.095915
North_America    0.575622
Japan            0.328463
Minimum variance portfolio annualized std deviation: 0.15289597855105871
Lowest component annualized std deviation 0.1690650344540889 ( North_America )


## Code segment 10.6
Figure 10.2 in Section 10.3

In [7]:
#Code Segment 10.6
#Generate graph of either simulated or historical sample correlation
#from df_logs

def make_corr_plot(df_logs, rtrial, samplesize, title_str, simulate):
#Generate a multivariate normal distribution using the data in df_logs
#compute sample correlations of size samplesize and graph them
#simulate: False, use historical data in df_logs
#          True, use simulated data in rtrial
    nobs=len(df_logs)
    corr_matrix=df_logs[df_logs.columns].corr()

    #Get sample correlations
    if simulate:
        samplecorrs=[np.corrcoef(rtrial[i-samplesize:i].transpose()) \
                    for i in range(samplesize,nobs+1)]
    else:
        samplecorrs=[df_logs.iloc[i-samplesize:i][df_logs.columns].corr().values \
                    for i in range(samplesize,nobs+1)]
    sccol=['r','g','b']
    stride=int((nobs-periodicity+1)/(4*periodicity))*periodicity
    
    dates=df_logs.index[samplesize-1:]
    plot_corrs(dates,samplecorrs,corr_matrix,sccol,stride, \
        title_str+str(samplesize)+'-week sample correlations')
#Done with make_corr_plot
    
def plot_corrs(dates,corrs,corr_matrix,sccol,stride,title_str):
    #dates and corrs have same length
    #dates in datetime format
    #corrs is a list of correlation matrices
    #corr_matrix has the target correlations
    #names of securities are the column names of corr_matrix
    #sccol is colors for lines
    #stride is how many dates to skip between ticks on x-axis
    #title_str is title string

    nobs=len(corrs)
    nsecs=len(corrs[0])

    #plot correlations in corrs, nsec per time period
    ncorrs=nsecs*(nsecs-1)/2
    z=0
    #Go through each pair
    for j in range(nsecs-1):
        for k in range(j+1,nsecs):
            #form time series of sample correlations
            #for this pair of securities
            cs=[corrs[i][j,k] for i in range(nobs)]
            plt.plot(range(nobs),cs, \
                     label=corr_matrix.columns[j]+'/'+ \
                     corr_matrix.columns[k], \
                     color=sccol[z])
            #Show target correlation in same color
            line=[corr_matrix.iloc[j,k]]*(nobs)
            plt.plot(range(nobs),line,color=sccol[z])
            z+=1

    plt.legend()
    tix=[x.strftime("%Y-%m-%d") for x in dates[0:nobs+1:stride]]
    plt.xticks(range(0,nobs+1,stride),tix,rotation=45)
    plt.ylabel("Correlation")
    plt.title(title_str)
    plt.grid()
    plt.show();
#Done with plot_corrs
         
#Generate a simulation    
#Show the Cholesky decomposition of the matrix
chol=np.linalg.cholesky(corr_matrix)
print('Cholesky:\n',chol)

#Generate random draws
nobs=len(df_logs)
nsecs=len(df_logs.columns)
strial=np.random.normal(0,1,size=[nobs,nsecs])
rtrial=np.matmul(chol,strial.T).T

samplesize=periodicity
title_str="Figure 10.2: Simulated "
simulate=True
import matplotlib.pyplot as plt
make_corr_plot(df_logs, rtrial, samplesize, title_str, simulate)
print("Standard Error = ",np.tanh(1/np.sqrt(periodicity-3)))

Cholesky:
 [[1.         0.         0.        ]
 [0.76164372 0.64799602 0.        ]
 [0.51049638 0.03638322 0.85910984]]


Standard Error =  0.14189319376693255


## Code segment 10.7
Figure 10.3 in Section 10.3

In [8]:
#Code Segment 10.7
#Get sample 3-year simulated correlations
samplesize=156
title_str="Figure 10.3: Simulated "
simulate=True
make_corr_plot(df_logs, rtrial, samplesize, title_str, simulate)
print("Standard Error = ",np.tanh(1/np.sqrt(156-3)))

Standard Error =  0.08066953425829818


## Code segment 10.8
Figure 10.4 in Section 10.3

In [9]:
#Code Segment 10.8
#Get sample 3-year historical correlations
samplesize=156
title_str="Figure 10.4: Historical "
simulate=False
make_corr_plot(df_logs, rtrial, samplesize, title_str, simulate)

## Code segment 10.9
Figure 10.5 in Section 10.4

In [10]:
#Code Segment 10.9
#Get FHFA house price index and graph it
df_fhfa=pd.read_excel( \
        "https://www.fhfa.gov/DataTools/Downloads/Documents/HPI/HPI_PO_summary.xls", \
        skiprows=1)

#Plot seasonally adjusted index
seasonal_index="Seasonally-Adjusted Purchase-Only Index \n(1991Q1=100)"
plt.plot(df_fhfa["Year"],df_fhfa[seasonal_index])
str_title="Figure 10.5: Seasonally Adjusted US House Price Index\n"
str_title+=str(df_fhfa.iloc[0]["Year"])+"Q"+str(df_fhfa.iloc[0]["Quarter"])+"-"
str_title+=str(df_fhfa.iloc[-1]["Year"])+"Q"+str(df_fhfa.iloc[-1]["Quarter"])
plt.ylabel("HPI")
plt.title(str_title)
plt.grid()
plt.show();

## Code segment 10.10
Figure 10.6 in Section 10.4

In [11]:
#Code Segment 10.10
import pandas as pd
import qrpm_funcs as qf
import datetime as dt
from datetime import datetime
import scipy.stats as spst
import numpy as np
import matplotlib.pyplot as plt

firstday="1970-12-31"
lastday=qf.LastYearEnd()
seriesnames=['WILL5000IND','DGS10']
cdates,ratematrix=qf.GetFREDMatrix(seriesnames,
                        startdate=firstday,enddate=lastday)

#Put in dataframe - no na's, only monthends
df_stockbond=pd.DataFrame(ratematrix,index=[datetime.strptime(cd,"%Y-%m-%d") \
                for cd in cdates]).dropna().resample('M').last()

#Convert to log-returns
#Assume Treasury rates given are for 10-year par bond
#paying semiannually
stock_logs=[]
bond_logs=[]
idx_stock=0
idx_tsy=1
T20=20.    #Number of semiannual periods in the 10-year bond
oldrate=df_stockbond.iloc[0][idx_tsy]/200    #Semiannual rate
for i in range(len(df_stockbond)-1):
    stock_logs.append(np.log(df_stockbond.iloc[i+1][idx_stock]/ \
                             df_stockbond.iloc[i][idx_stock]))
    #Use Formula 3.7 to get the duration of the par bond in semiannual periods
    duration=qf.formula3p7(oldrate*100.,oldrate*100.,T20)
    newrate=df_stockbond.iloc[i+1][idx_tsy]/200
    #Return for the month: carry plus principal change
    bond_return=oldrate/6-duration*(newrate-oldrate)
    bond_logs.append(np.log(1+bond_return))
    oldrate=newrate

#bring dates back to strings    
dates=[datetime.strftime(df_stockbond.index[i],"%Y-%m-%d") \
       for i in range(1,len(df_stockbond))]

corrs=[]
stride=36
for i in range(len(stock_logs)-stride):
    rho,p=spst.pearsonr(stock_logs[i:i+stride],bond_logs[i:i+stride])
    corrs.append(rho)

plt.plot(dates[stride:],corrs)
nobs=len(corrs)
stride_ticks=120
plt.xticks(range(0,nobs+1,stride_ticks), \
           dates[stride:-1:stride_ticks], \
           rotation=45)

#Shade insignificant area
fsig=1/np.sqrt(stride-3)
rsig=(np.exp(2*fsig)-1)/(np.exp(2*fsig)+1)

low_sig=[-np.abs(rsig)]*nobs
high_sig=[np.abs(rsig)]*nobs

plt.fill_between(range(nobs), low_sig, high_sig, \
                 facecolor='gray', alpha=0.5, interpolate=True)

str_title="Figure 10.6: US Stock/Bond "+str(stride)+ \
          "-month correlations\n"
str_title+=dates[stride]+" to "+dates[-1]
str_title+="   (Gray=insignificant)"
plt.ylabel("Correlation")
plt.title(str_title)
plt.grid()
plt.show();

## Code segment 10.11
Figure 10.7 in Section 10.5

In [12]:
#Code Segment 10.11
#Read current holdings of SPY Exchange-Traded-Fund (ETF)
#This is very close to the actual S&P 500 index
#There is a time mismatch - this is current while options information is from
#yearend, so implied correlations will be off a little
import pandas as pd
import time
import datetime
import numpy as np

#Function used below to get implied volatilities
def find_implieds(df_ATM,quote_date,day_target):
    #find options in dataframe df_ATM having maturities close to
    #day_target days from quote_date. Return average implieds of those
    #options and a string giving one of the expiration dates used.
    this_implied=0
    num_implied=0
    #Keep increasing the number of days interval
    #around day_target until some options are found
    for interval in range(16,86,10):
        for i_ATM in range(len(df_ATM)):
            if abs( int( \
             (df_ATM.iloc[i_ATM].expiration-quote_date) \
             /np.timedelta64(1, 'D'))-day_target)<interval:
                this_implied+=df_ATM.iloc[i_ATM].implied_volatility_1545
                num_implied+=1
                str_exp = str(df_ATM.iloc[i_ATM].expiration)[:10]
        if num_implied != 0:
            break

    return(this_implied/num_implied,str_exp)
#Done with find_implieds

#
# Step 1 - get SPY tickers
#
start_time = time.perf_counter()
url="https://www.ssga.com/us/en/individual/etfs/library-content/"+\
    "products/fund-data/etfs/us/holdings-daily-us-en-spy.xlsx"
df_spy=pd.read_excel(url,skiprows=4,engine="openpyxl")
df_spy.dropna(thresh=3,inplace=True)   #Clean up junk at the end
#Extract tickers and weights
orig_spy_tickers=[df_spy["Ticker"].iloc(0)[i].upper() for i in range(len(df_spy))]
weights=[df_spy["Weight"].iloc(0)[i] for i in range(len(df_spy))]
weights/=sum(weights)    #force weights to add to 1
end_time = time.perf_counter()
print("%d SPY tickers input, %7.3f seconds" % (len(weights),end_time-start_time))

#
# Step 2 - get options on the tickers in SPY
# and compute ATM volatility about 75 days out
#
yearend_string=qf.LastYearEnd()[:4]
df_spy_options=pd.read_excel(r"SPY_UnderlyingOptionsEODCalcs_"+ \
                             yearend_string+"-12-31.xlsx",engine="openpyxl")
end_time, start_time = time.perf_counter(), end_time
print("%d SPY options input, %7.3f seconds" % (len(df_spy_options), end_time-start_time))

#Match up stock tickers with options
spy_tickers=[]
implieds=[]
vis=[]   #vi's as in (10.16)
for i in range(len(orig_spy_tickers)):
    df_target=df_spy_options[df_spy_options.underlying_symbol == orig_spy_tickers[i]]
    if len(df_target)>0:   #Did we find any options on this ticker?
        spy_tickers.append(orig_spy_tickers[i])
        this_price = df_target.active_underlying_price_1545.unique()[0]
        quote_date = df_target.quote_date.unique()[0]
        #Find closest to the money
        min_to_money = min(abs(df_target.strike.unique()-this_price))
        df_ATM=df_target[abs(df_target.strike-this_price)==min_to_money]
        
        this_avg, str_exp = find_implieds(df_ATM,quote_date,75)
        implieds.append(this_avg)
        vis.append(implieds[-1]*weights[i])

end_time, start_time = time.perf_counter(), end_time
print("%d SPY tickers matched with options, %7.3f seconds" % (len(spy_tickers), \
                                                end_time-start_time))

#
# Step 3: Get SPX implied volatility about 75 days out
#
df_spx_options=pd.read_excel(r"SPX_UnderlyingOptionsEODCalcs_"+ \
                             yearend_string+"-12-31.xlsx", \
                             engine="openpyxl")
end_time, start_time = time.perf_counter(), end_time
print("%d SPX options input, %7.3f seconds" % (len(df_spx_options), \
                                               end_time-start_time))
#Get SPX ATM volatility
this_price = df_spx_options.active_underlying_price_1545.unique()[0]
#Find closest to the money
min_to_money = min(abs(df_spx_options.strike.unique()-this_price))
df_ATM=df_spx_options[abs(df_spx_options.strike-this_price)==min_to_money]

spx_ATM_60to90_implied, str_exp = find_implieds(df_ATM,quote_date,75)

#Make histogram of implieds

n, bins, patches = plt.hist(implieds,bins=50)
plt.axvline(spx_ATM_60to90_implied, color='k', linestyle='dashed', linewidth=1)
plt.annotate('S&P 500 implied vol', xy=(spx_ATM_60to90_implied, .9*max(n)), 
            xytext=(np.average(implieds)*1.5, .9*max(n)), va='center', 
            color='k', arrowprops=dict(color='k',width=1,headwidth=4))
plt.annotate('Average implied vol', xy=(np.average(implieds), .7*max(n)), 
            xytext=(np.average(implieds)*1.5, .7*max(n)), va='center',
            color='r', arrowprops=dict(color='r',width=1,headwidth=4))

plt.axvline(np.average(implieds), color='r', linestyle='dashed', linewidth=1)
plt.xlabel("Annualized Implied Vol")
plt.ylabel("Count ("+str(len(implieds))+" total)")
plt.title("Figure 10.7: Histogram of ATM implied vols of SPY components\n Expiring " \
          +str_exp+", Quoted "+str(quote_date)[:10])
plt.grid()
plt.show()

#Show number greater than SPX implied
n_greater = sum(iv > spx_ATM_60to90_implied for iv in implieds)
print("SPX implied:",spx_ATM_60to90_implied)
print("Number greater:",n_greater)
print("Average implied:",np.average(implieds))

#Do calculation for formula 10.16
var_port=spx_ATM_60to90_implied**2
sum_vi2=sum([vi**2 for vi in vis])
sum_vi_2=sum(vis)**2
implied_corr=(var_port-sum_vi2)/(sum_vi_2-sum_vi2)
print("Implied correlation per formula (10.16):",implied_corr)

504 SPY tickers input,   1.195 seconds


48618 SPY options input,  14.071 seconds


472 SPY tickers matched with options,   2.412 seconds


7612 SPX options input,   2.277 seconds


SPX implied: 0.15005000000000002
Number greater: 472
Average implied: 0.2865621115819209
Implied correlation per formula (10.16): 0.3156811774122457


## Code segment 10.12
Figure 10.8 in Section 10.5

In [13]:
#Code Segment 10.12
## Graph CBOE COR3N series
## https://www.cboe.com/us/indices/dashboard/cor3m/
## Captured February 26, 2022
import pandas as pd

df_cor3m=pd.read_excel("cboe_cor3m.xlsx",engine="openpyxl")
plt.plot(range(len(df_cor3m)),df_cor3m["COR3M"])
plt.grid()
plt.title('Figure 10.8: CBOE implied correlations COR3M')
nobs=len(df_cor3m)
stride=36
plt.xticks(range(0,nobs,stride),df_cor3m["Date"].iloc[0:nobs:stride])
plt.xlim(0,nobs)
plt.ylabel("Correlation (percent)")
#plt.ylim(25,90)
plt.show()

## Code segment 10.13
Figure 10.9 in Section 10.8.1

In [14]:
#Code Segment 10.13
import scipy.stats as spst
from scipy.optimize import minimize_scalar

#CHEAT! - get overall mean and standard deviation vectors
#In practice, would need to do everything out of sample - 
#start with a learning sample, e.g.
overallmean=np.mean(df_logs,axis=0)
overallstd=np.std(df_logs)
tickerlist=df_logs.columns

#Get GARCH params for each ticker
initparams=[.12,.85,.6]
gparams=[qf.Garch11Fit(initparams,df_logs[ticker]) for ticker in tickerlist]

minimal=10**(-20)
stgs=[] #Save the running garch sigmas
for it,ticker in enumerate(tickerlist):
    a,b,c=gparams[it]
    
    #Create time series of sigmas
    t=len(df_logs[ticker])
    stdgarch=np.zeros(t)
    stdgarch[0]=overallstd[ticker]
    #Compute GARCH(1,1) stddev's from data given parameters
    for i in range(1,t):
        #Note offset - i-1 observation of data
        #is used for i estimate of std deviation
        previous=stdgarch[i-1]**2
        var=c+b*previous+\
            a*(df_logs[ticker][i-1]-overallmean[ticker])**2
        stdgarch[i]=np.sqrt(var)

    #Save for later de-GARCHing
    stgs.append(stdgarch)
    stdgarch=100*np.sqrt(periodicity)*stgs[it]  #Annualize
    plt.plot(range(len(stdgarch)),stdgarch,label=ticker)
    
plt.grid()
plt.title('Figure 10.9: GARCH(1,1) annualized standard deviations '+ \
          min(df_logs.index.strftime("%Y%m"))+'-'+ \
          str(max(df_logs.index.strftime("%Y%m"))))
plt.ylabel('GARCH SDs')
plt.legend()
stride=5*periodicity
tix=[x.strftime("%Y-%m-%d") for x in df_logs.index[0:len(df_logs)-1:stride]]
plt.xticks(range(0,len(df_logs),stride),tix,rotation=45)
plt.show()

for it,ticker in enumerate(tickerlist):
    print(ticker,'a=%1.4f' % gparams[it][0], \
               'b=%1.4f' % gparams[it][1], \
               'c=%1.8f' % gparams[it][2], \
               'AnnEquilibStd=%1.4f' % \
               np.sqrt(periodicity*gparams[it][2]/ \
                       (1-gparams[it][0]-gparams[it][1])))

Europe a=0.1824 b=0.7684 c=0.00003543 AnnEquilibStd=0.1935
North_America a=0.1699 b=0.7815 c=0.00002997 AnnEquilibStd=0.1791
Japan a=0.1128 b=0.8367 c=0.00004145 AnnEquilibStd=0.2064


## Code segment 10.14
deGARCHed statistics in Section 10.8.1

In [15]:
#Code Segment 10.14
#Display before and after deGARCHing statistics

#Demeaned, DeGARCHed series go in dfeps
dfeps=df_logs.sort_values(by="Date").copy()
for it,ticker in enumerate(tickerlist):
    dfeps[ticker]-=overallmean[ticker]
    for i in range(len(dfeps)):
        dfeps[ticker].iloc[i]/=stgs[it][i]
    print(ticker)
    print('    DeGARCHed Mean:',np.mean(dfeps[ticker]))
    
    print('    Raw annualized Std Dev:',np.sqrt(periodicity)*overallstd[ticker])
    print('    DeGARCHed Std Dev:',np.std(dfeps[ticker]))
    
    print('    Raw excess kurtosis:',spst.kurtosis(df_logs[ticker]))
    print('    DeGARCHed Excess Kurtosis:',spst.kurtosis(dfeps[ticker]))

Europe
    DeGARCHed Mean: -0.006335465072757949
    Raw annualized Std Dev: 0.185716323601916
    DeGARCHed Std Dev: 1.000934838881549
    Raw excess kurtosis: 5.72308609689704
    DeGARCHed Excess Kurtosis: 1.5791736576196476


North_America
    DeGARCHed Mean: -0.014069088834356223
    Raw annualized Std Dev: 0.16901518481878622
    DeGARCHed Std Dev: 1.0002445570822687
    Raw excess kurtosis: 5.580467633225819
    DeGARCHed Excess Kurtosis: 2.500713051644463
Japan
    DeGARCHed Mean: -0.0112808838314883
    Raw annualized Std Dev: 0.20210680336454045


    DeGARCHed Std Dev: 1.0011833061949984
    Raw excess kurtosis: 2.1528985498298443
    DeGARCHed Excess Kurtosis: 1.7804141414949157


## Code segment 10.15
Figure 10.10 in Section 10.8.1

In [16]:
#Code Segment 10.15
#Get sample 156-month de-GARCHED correlations
samplesize=3*periodicity
title_str="Figure 10.10: Historical de-GARCHed "
simulate=False
make_corr_plot(dfeps, rtrial, samplesize, title_str, simulate)

## Code segment 10.16
Figure 10.11 in Section 10.8.2

In [17]:
#Code Segment 10.16
#Compute integrated correlations

InData=np.array(dfeps[tickerlist])

def IntegratedCorrObj(s):
    #Compute time series of quasi-correlation
    #matrices from InData using integrated parameter
    #xlam=exp(s)/(1+exp(s)); note this format removes
    #the need to enforce bounds of xlam being between
    #0 and 1. This is applied to formula 10.29.
    #Standardize Q's and apply formula 10.34.
    #Returns scalar 10.34
    xlam=np.exp(s)
    xlam/=1+xlam
    obj10p34=0.
    previousq=np.identity(len(InData[0]))
    #Form new shock matrix
    for i in range(len(InData)):
        #standardize previous q matrix
        #and compute contribution to objective
        #function
        stdmtrx=np.diag([1/np.sqrt(previousq[s,s]) for s in range(len(previousq))])
        previousr=stdmtrx @ (previousq @ stdmtrx)
        #objective function
        obj10p34+=np.log(np.linalg.det(previousr))
        shockvec=np.array(InData[i])
        vec1=shockvec @ np.linalg.inv(previousr)
        #This makes obj10p34 into a 1,1 matrix
        obj10p34+=vec1 @ shockvec
              
        #Update q matrix
        shockvec=np.mat(shockvec)
        shockmat=np.matmul(shockvec.T,shockvec)
        previousq=xlam*shockmat+(1-xlam)*previousq
    return(obj10p34[0,0])
#Done with IntegratedCorrObj

result=minimize_scalar(IntegratedCorrObj)

xlamopt=np.exp(result.x)
xlamopt/=1+xlamopt
print('Optimal lambda:',xlamopt)
print('Optimal objective function:',result.fun)
if xlamopt>=1 or xlamopt==0:
    halflife=0
else:
    halflife=-np.log(2)/np.log(1-xlamopt)
print('Half-life (years):',halflife/periodicity)

#Compute integrated correlations
nobs=len(InData)
nsecs=len(InData[0])
#Start quasi-correlation matrix series with identity
previousq=np.identity(nsecs)
rmatrices=[]
for i in range(nobs):
    stdmtrx=np.diag([1/np.sqrt(previousq[s,s]) for s in range(nsecs)])
    rmatrices.append(np.matmul(stdmtrx,np.matmul(previousq,stdmtrx)))
    shockvec=np.mat(np.array(InData[i]))
    #Update q matrix
    shockmat=np.matmul(shockvec.T,shockvec)
    previousq=xlamopt*shockmat+(1-xlamopt)*previousq

#Plot integrated correlations
iccol=['r','g','b']
xtitle='Figure 10.11: Integrated correlations λ=%1.5f' % xlamopt
xtitle+=', '+min(df_logs.index.strftime("%Y-%m-%d"))+':'+ \
                  max(df_logs.index.strftime("%Y-%m-%d"))
dates=df_logs.index
stride=5*periodicity
plot_corrs(dates,rmatrices,corr_matrix,iccol,stride,xtitle)

Optimal lambda: 0.011849775161762383
Optimal objective function: 3346.489490908641
Half-life (years): 1.1182169359574616


## Code segment 10.17
Figure 10.12 in Section 10.8.2

In [18]:
#Code Segment 10.17
#Map of objective with respect to half-life

halflife=int(halflife)
delta_halflife=int(halflife/2)

x=np.arange(halflife-delta_halflife,halflife+delta_halflife)
y=[IntegratedCorrObj(np.log((.5)**(-1/h)-1)) for h in x]

plt.plot(x/periodicity,y)
plt.title("Figure 10.12: Objective Function as half-life changes")
plt.xlabel("Halflife in years")
plt.ylabel("Objective")
plt.grid()
plt.show();

## Code segment 10.18
Figure 10.13 in Section 10.8.2

In [19]:
#Code Segment 10.18
def MeanRevCorrObj(params):
    #Compute time series of quasi-correlation
    #matrices from InData using mean reverting
    #formula 10.30. Standardize them and apply
    #formula 10.34. Returns scalar 10.34
    
    #Extract parameters
    alpha,beta=params
    #Enforce bounds
    if alpha<0 or beta<0:
        return(10**20)
    elif (alpha+beta)>.999:
        return(10**20)
    obj10p34=0
    #Initial omega is obtained through correlation targeting
    Rlong=np.corrcoef(InData.T)
    previousq=np.identity(len(InData[0]))
    #Form new shock matrix
    for i in range(len(InData)):
        #standardize previous q matrix
        #and compute contribution to objective
        #function
        stdmtrx=np.diag([1/np.sqrt(previousq[s,s]) \
                         for s in range(len(previousq))])
        previousr=np.matmul(stdmtrx,np.matmul(previousq,stdmtrx))
        #objective function
        obj10p34+=np.log(np.linalg.det(previousr))
        shockvec=np.array(InData[i])
        vec1=np.matmul(shockvec,np.linalg.inv(previousr))
        #This makes obj10p34 into a 1,1 matrix
        obj10p34+=np.matmul(vec1,shockvec)
              
        #Update q matrix
        shockvec=np.mat(shockvec)
        shockmat=np.matmul(shockvec.T,shockvec)
        previousq=(1-alpha-beta)*Rlong+alpha*shockmat+beta*previousq
    return(obj10p34[0,0])
#Done with MeanRevCorrObj

import scipy.optimize as scpo
#alpha and beta positive
corr_bounds = scpo.Bounds([0,0],[np.inf,np.inf])
#Sum of alpha and beta is less than 1
corr_linear_constraint = \
    scpo.LinearConstraint([[1, 1]],[0],[.999])

initparams=[.02,.93]

results = scpo.minimize(MeanRevCorrObj, \
        initparams, \
        method='trust-constr', \
        jac='2-point', \
        hess=scpo.SR1(), \
        bounds=corr_bounds, \
        constraints=corr_linear_constraint)

alpha,beta=results.x
print('Optimal alpha, beta:',alpha,beta)
print('Optimal objective function:',results.fun)
halflife=-np.log(2)/np.log(beta)
print('Half-life (years):',halflife/periodicity)

#Compute mean reverting correlations
nobs=len(InData)
nsecs=len(InData[0])
previousq=np.identity(nsecs)
Rlong=np.corrcoef(InData.T)
rmatrices=[]
for i in range(nobs):
    stdmtrx=np.diag([1/np.sqrt(previousq[s,s]) for s in range(nsecs)])
    rmatrices.append(np.matmul(stdmtrx,np.matmul(previousq,stdmtrx)))
    shockvec=np.mat(np.array(InData[i]))
    #Update q matrix
    shockmat=np.matmul(shockvec.T,shockvec)
    previousq=(1-alpha-beta)*Rlong+alpha*shockmat+beta*previousq

#Plot mean-reverting correlations
iccol=['r','g','b']
xtitle='Figure 10.13: Mean Reverting Correlations α=%1.5f' % alpha
xtitle+=', β=%1.5f' % beta
xtitle+=',\n'+min(df_logs.index.strftime("%Y-%m-%d"))+':'+ \
             max(df_logs.index.strftime("%Y-%m-%d"))
dates=df_logs.index
stride=5*periodicity
plot_corrs(dates,rmatrices,corr_matrix,iccol,stride,xtitle)

/usr/local/lib/python3.10/dist-packages/scipy/optimize/_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


Optimal alpha, beta: 0.015336358982559405 0.9760831978440717
Optimal objective function: 3323.457027587737
Half-life (years): 0.5506466848231422


## Code segment 10.19
MacGyver method calculations in Section 10.8.3

In [20]:
#Code Segment 10.19
#MacGyver method - pairwise integrated
minimal=10**(-20)
xlams=[]
for it in range(len(tickerlist)-1):
    tick1=tickerlist[it]
    for jt in range(it+1,len(tickerlist)):
        tick2=tickerlist[jt]
        InData=np.array(dfeps[[tick1,tick2]])
        result=minimize_scalar(IntegratedCorrObj)
        xlamopt=np.exp(result.x)/(1+np.exp(result.x))
        print(tick1,tick2)
        print('    Optimal lambda:',xlamopt)
        print('    Optimal objective function:', \
              result.fun)
        if np.absolute(xlamopt)<minimal or xlamopt>=1:
            halflife=0
        else:
            halflife=-np.log(2)/np.log(1-xlamopt)
        print('    Half-life (years):',halflife/periodicity)
        xlams.append(xlamopt)

print('\nMedian MacGyver lambda:',np.median(xlams))

Europe North_America
    Optimal lambda: 0.030413043920421285
    Optimal objective function: 2156.7933064578374
    Half-life (years): 0.4315914887152863


Europe Japan
    Optimal lambda: 0.010313609686724502
    Optimal objective function: 2875.2528154492807
    Half-life (years): 1.2857666875857823


North_America Japan
    Optimal lambda: 0.0068378704058871895
    Optimal objective function: 3054.6117169028635
    Half-life (years): 1.9427288036119008

Median MacGyver lambda: 0.010313609686724502
